<a href="https://colab.research.google.com/github/patshandofdoom/Pauper-Deck-list-Scraping/blob/main/Pauper_Deck_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Features to add:
*  Review script and see if any optimizations can be made
* Evaluate downloading other decklists from Wizards
* evaluate downloading other Decklists from

# Gathering info

First we need to install all the functions needed to make this scraper work. Beautiful soup is key here as it allows us to search the HTML for our

In [9]:
from html import escape
from html import unescape
from bs4 import BeautifulSoup
import requests
import re
import json
from datetime import datetime, timedelta, date
from dateutil import parser
from google.colab import drive


Next we define all the functions that handle the information. This section could use some work, but it functions right now sooooo...


In [2]:
#Defines a function that returns event ditionary, which is the building block of our program

def event_define(deck_link_tail):
  #Build the link for the event
  deck_link = 'https://www.mtggoldfish.com'+deck_link_tail+'#paper'

  #Get the gonctent from the page and parse the HTML
  deck_page = requests.get(deck_link, headers={"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"})
  deck_page_soup = BeautifulSoup(deck_page.content, "html.parser")

  #Find the event name within the soup
  event_name = deck_page_soup.find('p', class_='deck-container-information').find_next('a').get_text()

  #Parse the event date from the event name using Regex
  event_date = re.sub('\s','',re.sub('[A-z]+','',event_name))

  #Build the event link using components from the HTML
  event_link = 'https://www.mtggoldfish.com'+deck_page_soup.find('p', class_='deck-container-information').find_next('a').get('href')

  #Create a dictionary for the event and return it
  event= {
        "Event Name":event_name,
        "Event Date":event_date,
        "Event Link":event_link,
        "Decks":{}
        }

  return event
##################################################
#Defines a function that returns the deck soup of the deck after grabbing it with beautifulsoup
def get_html(index, deck_link_tail):
  #Build the deck link
  deck_link = 'https://www.mtggoldfish.com'+deck_link_tail+'#paper'
  #send the request with the proper headers to get a response
  deck_page = requests.get(deck_link, headers={"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"})
  #Convert the response into a searchable object with beautifulsoup
  deck_page_soup = BeautifulSoup(deck_page.content, "html.parser")

  #return the deck page soup so that we dont need to call the HTML again in the next function
  return(deck_page_soup)

##################################################
#Create a deck info function in order to simplify the 'for loop' bullshit and make it more clear

def deck_info_function(deck_page_soup, deck_link_tail, event,index):
  #remove spaces and linebreaks
  deck_link = 'https://www.mtggoldfish.com'+deck_link_tail+'#paper'
  deck_name = re.sub('\n',"",deck_page_soup.find('h1', class_='title').contents[0])
  pilot = re.sub("^by\s","",deck_page_soup.find("span", class_="author").get_text())
  format = re.sub("[A-z]*:\s","",re.sub("\n","",deck_page_soup.find('p', class_='deck-container-information').contents[0]))

  archetype_container = deck_page_soup.find('p', class_='deck-container-information').find_all('a')
  #not all decks have an archetype designated so we need to check if it exists before we try and access it
  if((archetype_container is not None) and (len(archetype_container)>=3)):
    archetype = archetype_container[2].get_text()
    archetype_link = archetype_container[2].get("href")
  else:
    archetype = "Unlisted"
    archetype_link = "Unlisted"

  event["Decks"].update({"Deck "+str(index):
    {
    "DeckName":deck_name,
    "Pilot":pilot,
    "DeckLink":deck_link,
    "Format":format,
    "Archetype":archetype,
    "Archetype Link":"https://www.mtggoldfish.com"+archetype_link,
    "Mainboard":{},
    "Sideboard":{}
    }})
  return event

##################################################
def add_card(this_object, card_name,card_quantity,main_side):
  if main_side == "main":
    this_object["Mainboard"].update({card_name:card_quantity})
  else:
    this_object["Sideboard"].update({card_name:card_quantity})

##################################################
#Defines a function that returns the deck soup of the deck after grabbing it with beautifulsoup

def get_html(index, deck_link_tail):
  #Build the deck link
  deck_link = 'https://www.mtggoldfish.com'+deck_link_tail+'#paper'
  #send the request with the proper headers to get a response
  deck_page = requests.get(deck_link, headers={"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"})
  #Convert the response into a searchable object with beautifulsoup
  deck_page_soup = BeautifulSoup(deck_page.content, "html.parser")

  #return the deck page soup so that we dont need to call the HTML again in the next function
  return(deck_page_soup)

##################################################
#Create a deck info function in order to simplify the 'for loop' bullshit and make it more clear

def deck_info_function(deck_page_soup, deck_link_tail, event,index):
  #remove spaces and linebreaks
  deck_link = 'https://www.mtggoldfish.com'+deck_link_tail+'#paper'
  deck_name = re.sub('\n',"",deck_page_soup.find('h1', class_='title').contents[0])
  pilot = re.sub("^by\s","",deck_page_soup.find("span", class_="author").get_text())
  format = re.sub("[A-z]*:\s","",re.sub("\n","",deck_page_soup.find('p', class_='deck-container-information').contents[0]))

  archetype_container = deck_page_soup.find('p', class_='deck-container-information').find_all('a')
  #not all decks have an archetype designated so we need to check if it exists before we try and access it
  if((archetype_container is not None) and (len(archetype_container)>=3)):
    archetype = archetype_container[2].get_text()
    archetype_link = archetype_container[2].get("href")
  else:
    archetype = "Unlisted"
    archetype_link = "Unlisted"

  event["Decks"].update({"Deck "+str(index):
    {
    "DeckName":deck_name,
    "Pilot":pilot,
    "DeckLink":deck_link,
    "Format":format,
    "Archetype":archetype,
    "Archetype Link":"https://www.mtggoldfish.com"+archetype_link,
    "Mainboard":{},
    "Sideboard":{}
    }})
  return event

##################################################

def add_card(this_object, card_name,card_quantity,main_side):
  if main_side == "main":
    this_object["Mainboard"].update({card_name:card_quantity})
  else:
    this_object["Sideboard"].update({card_name:card_quantity})

##################################################
def decklist_add(deck_html,index, event):
  #Grab all card names and quantities from the decklist
  deck_table = deck_html.find("table",class_="deck-view-deck-table")
  deck_rows = deck_table.find_all("tr")
  #Set a switch that will let the program know where to store the card, either main-deck or sideboard
  main_side_switch = 'main'

  #For each of the rows, get the card name, card quantity, then add it to the deck
  for each_row in deck_rows:
    if each_row.find("th") is None:
      this_card_name = each_row.find_next("span",class_="card_id card_name").get_text()
      this_card_quantity = re.sub("\n","",each_row.find_next("td",class_="text-right").get_text())
      add_card(event["Decks"]["Deck "+str(index)],this_card_name,this_card_quantity,main_side_switch)
    #If one of hte rows contains the sideboard header, change the switch so that the cards are added to the sideboard
    else:
      header_value = each_row.find("th").get_text()
      if re.search('Sideboard',header_value):
        main_side_switch = 'side'

##################################################

def compile_event_info(link_to_event):
  #Script commands actually start here, We start by grabbing the pauper league overview page
  page = requests.get(link_to_event)
  page_soup = BeautifulSoup(page.content, "html.parser")

  #Get each <tr> seperately into an array
  #Divide all the <td>'s into arrays, grab the href from the <a> object from each of the second TD's
  #This should give the link to each deck: "https://www.mtggoldfish.com" + href


  deck_links = []
  rows_for_tourney = page_soup.find("table",class_="table-tournament").find_all("tr")

  for each_row in rows_for_tourney:
    table_cells = each_row.find_all("td")
    if len(table_cells) > 2:
      deck_links.append(table_cells[1].find_next('a').get('href'))

  #Need to establish a list of decklists for each event where we can push the deck info
  decklists_for_event = []
  event = event_define(deck_links[0])

  for index, each_deck_link in enumerate(deck_links):
    deck_html = get_html(index, each_deck_link)
    event = deck_info_function(deck_html,each_deck_link,event,index)
    decklist_add(deck_html,index, event)
    this_deck = event["Decks"]["Deck "+str(index)]

  print("All Decks Processed")
  return event
##################################################

def compile_each_event(this_event_link):
  #full_event_dictionary = compile_event_info("https://www.mtggoldfish.com/tournament/pauper-league-2022-07-27#paper")
  full_event_dictionary = compile_event_info(this_event_link)
  return(full_event_dictionary)

##################################################


<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:49: SyntaxWarning: invalid escape sequence '\s'
<>:50: SyntaxWarning: invalid escape sequence '\s'
<>:102: SyntaxWarning: invalid escape sequence '\s'
<>:103: SyntaxWarning: invalid escape sequence '\s'
<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:49: SyntaxWarning: invalid escape sequence '\s'
<>:50: SyntaxWarning: invalid escape sequence '\s'
<>:102: SyntaxWarning: invalid escape sequence '\s'
<>:103: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2458001137.py:15: SyntaxWarning: invalid escape sequence '\s'
  event_date = re.sub('\s','',re.sub('[A-z]+','',event_name))
/tmp/ipython-input-2458001137.py:49: SyntaxWarning: invalid escape sequence '\s'
  pilot = re.sub("^by\s","",deck_page_soup.find("span", class_="author").get_text())
/tmp/ipython-input-2458001137.py:50: SyntaxWarning: invalid escape sequence '\s'
  format = re.sub("[A-z]*:\s","",re.sub("\n","",deck_page_soup.find('p', class_='deck-container-

# Storing to Google Sheet

This section will process each of th relevant URLs in the returned JSON, then will call each of the relevant functions from the second block to process the infom from the URL, finally sending it back to the Google spreasheet API to be stored

In [3]:
from IPython.core.prefilter import EmacsHandler
#Get the list of links from the stored file Website Check Dicitonary
filepath = "/content/drive/MyDrive/Colab Notebooks/Pauper Deck Analysis 3.0/Website Check Dictionary"
save_dictionary_writepath = "/content/drive/MyDrive/Colab Notebooks/Pauper Deck Analysis 3.0/AWS Write File"


with open(filepath,'r') as readfile:
  status_dictionary = readfile.read()
  status_dictionary = json.loads(status_dictionary)

print(len(status_dictionary))

save_dictionary = {}
for each_record in status_dictionary:
  print(each_record)
  current_record = status_dictionary[each_record]

  #Sort through the ones where index [2] = "Good" and index[3] = "not stored"
  if current_record[2] != "Good" or current_record == "stored":
    status_dictionary.pop(each_record)
  else:
    event_dictionary = compile_each_event(current_record[1])
    save_dictionary[each_record] = event_dictionary
    print(event_dictionary)
    with open(save_dictionary_writepath, 'w') as writefile:
      writefile.write(json.dumps(save_dictionary))
      writefile.close()

#
with open(save_dictionary_writepath,'r') as readfile:
  print(json.loads(readfile.read()))


#Need to store all the events in a new dictionary and the write them to a new file


0
{'Pauper League 2022-09-07': {'Event Name': 'Pauper League 2022-09-07', 'Event Date': '2022-09-07', 'Event Link': 'https://www.mtggoldfish.com/tournament/40498', 'Decks': {'Deck 0': {'DeckName': 'Dimir Control', 'Pilot': 'abbadon55', 'DeckLink': 'https://www.mtggoldfish.com/deck/5079399#paper', 'Format': 'Pauper', 'Archetype': 'Dimir Control', 'Archetype Link': 'https://www.mtggoldfish.com/archetype/pauper-dimir-control-1e2279c1-d135-4609-8b99-83f2ab98c424', 'Mainboard': {'Augur of Bolas': '4', 'Aarakocra Sneak': '4', 'Vicious Battlerager': '4', 'Gurmag Angler': '2', 'Brainstorm': '3', 'Dark Ritual': '4', 'Preordain': '3', 'Agony Warp': '1', 'Cast Down': '3', 'Counterspell': '2', 'Exclude': '1', 'Suffocating Fumes': '1', 'Foil': '4', 'Snuff Out': '4', 'Contaminated Aquifer': '4', 'Ice Tunnel': '4', 'Snow-Covered Island': '10', 'Snow-Covered Swamp': '2'}, 'Sideboard': {'Annul': '2', 'Blue Elemental Blast': '3', 'Dispel': '1', 'Relic of Progenitus': '2', 'Agony Warp': '1', "Chainer's E

In [4]:
with open(save_dictionary_writepath,'r') as readfile:
  this_file = json.loads(readfile.read())
  print(this_file)
  print(len(this_file))

{'Pauper League 2022-09-07': {'Event Name': 'Pauper League 2022-09-07', 'Event Date': '2022-09-07', 'Event Link': 'https://www.mtggoldfish.com/tournament/40498', 'Decks': {'Deck 0': {'DeckName': 'Dimir Control', 'Pilot': 'abbadon55', 'DeckLink': 'https://www.mtggoldfish.com/deck/5079399#paper', 'Format': 'Pauper', 'Archetype': 'Dimir Control', 'Archetype Link': 'https://www.mtggoldfish.com/archetype/pauper-dimir-control-1e2279c1-d135-4609-8b99-83f2ab98c424', 'Mainboard': {'Augur of Bolas': '4', 'Aarakocra Sneak': '4', 'Vicious Battlerager': '4', 'Gurmag Angler': '2', 'Brainstorm': '3', 'Dark Ritual': '4', 'Preordain': '3', 'Agony Warp': '1', 'Cast Down': '3', 'Counterspell': '2', 'Exclude': '1', 'Suffocating Fumes': '1', 'Foil': '4', 'Snuff Out': '4', 'Contaminated Aquifer': '4', 'Ice Tunnel': '4', 'Snow-Covered Island': '10', 'Snow-Covered Swamp': '2'}, 'Sideboard': {'Annul': '2', 'Blue Elemental Blast': '3', 'Dispel': '1', 'Relic of Progenitus': '2', 'Agony Warp': '1', "Chainer's Edi

# Re-write of Google Sheets Functionality

This section of blocks is a re-writre of the google sheets sheet where we generate the URLs for the events and check the validity of those links

In [5]:
#Check if website is live by jsut downloading the header instead of the entire HTML
#https://stackoverflow.com/questions/16778435/python-check-if-website-exists

def check_site(url_to_check):
  response = requests.get('http://www.example.com')
  if response.status_code == 200:
      return ("Good")
  else:
      return("Bad")



This block is used to iterate through old dates to see if a pauper league event exists. It checks the validity of these links using the HTML checker above.

In [6]:
start_period = "2015-01-07"
#Convert to strptime in order to compare to today
start_period = datetime.strptime(start_period,'%Y-%m-%d')
current_date = datetime.today()
event_dictionary = {}
filepath = "/content/drive/MyDrive/Colab Notebooks/Pauper Deck Analysis 3.0/Website Check Dictionary"

#open the readfile to see if it contains the current date
with open(filepath,'r') as readfile:
  readdictionary = readfile.read()

#check if readdictionary is blank. If not blank, load the JSON. if yes blank, create a dictionary
if readdictionary is not "":
  readdictionary = json.loads(readdictionary)
  #event_dictionary = readdictionary
else:
  readdictionary = {}

#for each week leading up to this one
while start_period < current_date:
  #build the website url from the date
  event_datecode = start_period.strftime('%Y-%m-%d')
  event_URL = "https://www.mtggoldfish.com/tournament/pauper-league-"+event_datecode+"#paper"

  #create a variable to track if the date has already been checked
  if event_datecode in readdictionary.keys():
    already_in_use = "yes"
  else:
    already_in_use = "no"

  if already_in_use == "no":
    site_status = check_site(event_URL)
    event_code = "Pauper League "+event_datecode
    storage_status = "not stored"
    readdictionary[event_code] = [event_datecode,event_URL,site_status,storage_status]

  #adding days to a date
  #https://stackoverflow.com/questions/6871016/adding-days-to-a-date-in-python
  end_date = start_period + timedelta(days=7)
  start_period = end_date

print(readdictionary)

<>:13: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
<>:13: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
/tmp/ipython-input-606175303.py:13: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
  if readdictionary is not "":


{'Pauper League 2015-01-07': ['2015-01-07', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-07#paper', 'Good', 'not stored'], 'Pauper League 2015-01-14': ['2015-01-14', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-14#paper', 'Good', 'not stored'], 'Pauper League 2015-01-21': ['2015-01-21', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-21#paper', 'Good', 'not stored'], 'Pauper League 2015-01-28': ['2015-01-28', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-28#paper', 'Good', 'not stored'], 'Pauper League 2015-02-04': ['2015-02-04', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-04#paper', 'Good', 'not stored'], 'Pauper League 2015-02-11': ['2015-02-11', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-11#paper', 'Good', 'not stored'], 'Pauper League 2015-02-18': ['2015-02-18', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-18#paper', 'Good', 'not stored'], 'Pauper League 2015-02-25'

This block saves the dictionary to the file on my google drive

In [7]:
with open(filepath, 'w') as writefile:
    writefile.write(json.dumps(readdictionary))
    #writefile.write("")
    writefile.close()


with open(filepath,'r') as readfile:
    readdictionary = readfile.read()
    readdictionary = json.loads(readdictionary)
    print(readdictionary)
    print(len(readdictionary))


{'Pauper League 2015-01-07': ['2015-01-07', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-07#paper', 'Good', 'not stored'], 'Pauper League 2015-01-14': ['2015-01-14', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-14#paper', 'Good', 'not stored'], 'Pauper League 2015-01-21': ['2015-01-21', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-21#paper', 'Good', 'not stored'], 'Pauper League 2015-01-28': ['2015-01-28', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-28#paper', 'Good', 'not stored'], 'Pauper League 2015-02-04': ['2015-02-04', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-04#paper', 'Good', 'not stored'], 'Pauper League 2015-02-11': ['2015-02-11', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-11#paper', 'Good', 'not stored'], 'Pauper League 2015-02-18': ['2015-02-18', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-18#paper', 'Good', 'not stored'], 'Pauper League 2015-02-25'

Test to see if removing a value from the file and dictionary will be fixed when I re-run the file.

In [8]:
#Default title text
with open(filepath, 'w') as writefile:
    key = "Pauper League 2016-10-12"
    if key in readdictionary.keys():
      readdictionary.pop(key)
    print(readdictionary)
    writefile.write(json.dumps(readdictionary))
    #writefile.write("")
    writefile.close()
#print(event_dictionary)


{'Pauper League 2015-01-07': ['2015-01-07', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-07#paper', 'Good', 'not stored'], 'Pauper League 2015-01-14': ['2015-01-14', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-14#paper', 'Good', 'not stored'], 'Pauper League 2015-01-21': ['2015-01-21', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-21#paper', 'Good', 'not stored'], 'Pauper League 2015-01-28': ['2015-01-28', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-01-28#paper', 'Good', 'not stored'], 'Pauper League 2015-02-04': ['2015-02-04', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-04#paper', 'Good', 'not stored'], 'Pauper League 2015-02-11': ['2015-02-11', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-11#paper', 'Good', 'not stored'], 'Pauper League 2015-02-18': ['2015-02-18', 'https://www.mtggoldfish.com/tournament/pauper-league-2015-02-18#paper', 'Good', 'not stored'], 'Pauper League 2015-02-25'

Link to the data studio account:

https://datastudio.google.com/u/0/reporting/7214b19c-cad6-44a8-9892-fbea5dc419b1/page/qWc0C/edit

Link to the Googls Sheet:

https://docs.google.com/spreadsheets/d/1d-X0qKTMXzWmUTLlmgjiHNuLPZdJ4CxrDiSdKQiQ_qE/edit#gid=771810328